In [12]:
## Hidden Markov Model
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('aggregated.csv')
trend_data = df['nvidia'].values
price_data = df['NVDA Monthly'].values
random.seed(42)


### hidden states will be good/bad/ some assumption or distribution model
### observable will be buying or selling or up or down (feature engineer it by compressing into 1 vector.)


## Hidden Markov Model
Using Bauch as our maximizer and Viterbi to decide states, we draft our plan as such: \

1. q_state = 3 { Buy, Hold, Sell }

2. hidden_states = { Buy, Hold, Sell }
2a. obs_states = { Up, Down, No Change }

3. transition matrix is defined as: \
    B>B(signal=2), B>H(signal=0), B>S(signal=-2), \
    H>B(signal=1), H>H(signal=0), H>S(signal=-1), \
    S>B(signal=2), S>H(signal=0), S>S(signal=-2)

4. emission matrix is defined as "Given state i, P(j)": \
    Up (0) = P(B|S|H), \
    Down (1) = P(B|S|H), \
    Stale (2) = P(B|S|H)
    

#### Matrix plan v1
Transition matrix:\


Emission matrix:\
if -0.1 < ROC < 0.1 : Stale \
if ROC > 0.1 : Up \
if ROC < -0.1 : Down




In [26]:
n_hidden = 3 # states: buy, hold, sell
n_obs = 3 # observations: up, down, no change
threshold = 0.05
total_trend, total_price = len(trend_data), len(price_data)

discrete_trend_data = [0 if x >= threshold else 1 if x <= -threshold else 2 for x in trend_data]
discrete_price_data = [0 if x >= threshold else 1 if x <= -threshold else 2 for x in price_data]

transition_matrix = np.array([[0.1, 0.2, 0.3],
                              [0.1, 0.2, 0.3],
                              [0.1, 0.2, 0.3]])
emission_matrix = np.array([[0.1, 0.2, 0.3],
                            [0.1, 0.2, 0.3],
                            [0.1, 0.2, 0.3]])
                            

##

In [27]:
### TESTING ###


# Counting the number of symbols
price_symbols = {'sell': 0, 'buy': 0, 'hold': 0}
trend_symbols = {'down': 0, 'up': 0, 'no change': 0}

for i in range(total_trend):
    trend_symbols['up'] += 1 if discrete_trend_data[i] == 0 else 0
    trend_symbols['down'] += 1 if discrete_trend_data[i] == 1 else 0
    trend_symbols['no change'] += 1 if discrete_trend_data[i] == 2 else 0

for i in range(total_price):
    price_symbols['buy'] += 1 if discrete_price_data[i] == 0 else 0
    price_symbols['sell'] += 1 if discrete_price_data[i] == 1 else 0
    price_symbols['hold'] += 1 if discrete_price_data[i] == 2 else 0


hidden_states = []
for trend_value, price_value in zip(discrete_trend_data, discrete_price_data):
    if (trend_value == 1 and price_value == 1) or (trend_value == 1 and price_value == 2):  # Trend indicates buy opportunity
        hidden_states.append("buy")
    elif (trend_value == 0 and price_value == 0) or (trend_value == 0 and price_value == 2):  # Trend indicates sell opportunity
        hidden_states.append("sell")
    else:  # Trend and price do not provide clear buy or sell signals
        hidden_states.append("hold")

hidden_states

['hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'buy',
 'sell',
 'sell',
 'buy',
 'hold',
 'buy',
 'buy',
 'hold',
 'sell',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'sell',
 'hold',
 'hold',
 'sell',
 'hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'hold',
 'sell',
 'buy',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'buy',
 'hold',
 'buy',
 'buy',
 'sell',
 'hold',
 'buy',
 'hold',
 'buy',
 'hold',
 'sell',
 'sell',
 'hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'hold',
 'buy',
 'sell',
 'buy',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'hold',
 'hold',
 'sell',
 'hold',
 'hold',
 'buy',
 'hold',
 'hold',
 'hold',
 'hold',
 'buy',
 'buy',
 'sell',
 'buy',
 'buy',
 'sell',
 'sell',
 'hold',
 'hold',
 'hold',
 'buy',
 'hold',
 'sell

In [15]:
# Building Trend Emission Matrix

# zip hidden (my decisions) emitting observable
# e.g. "buy" emitting "up"

# Initialize dictionaries to store symbol counts for each hidden state
trend_symbol_counts = {}
state_counts = {}

# Iterate over the time series data of trend and price data along with the corresponding hidden states
for state, trend_symbol, price_symbol in zip(hidden_states, discrete_trend_data, discrete_price_data):
    # Update symbol counts for trend data
    if state not in trend_symbol_counts:
        trend_symbol_counts[state] = {}
    if trend_symbol not in trend_symbol_counts[state]:
        trend_symbol_counts[state][trend_symbol] = 0
    trend_symbol_counts[state][trend_symbol] += 1
    
    # Update state counts
    if state not in state_counts:
        state_counts[state] = 0
    state_counts[state] += 1

# Calculate emission probabilities for trend data
trend_emission_probs = {}
for state in trend_symbol_counts:
    trend_emission_probs[state] = {}
    total_count = state_counts[state]
    for trend_symbol, count in trend_symbol_counts[state].items():
        trend_emission_probs[state][trend_symbol] = count / total_count

# Each row corresponds to a hidden state, and each column corresponds to a discrete symbol
trend_emission_matrix = np.array([[trend_emission_probs[state].get(symbol, 0) for symbol in set(discrete_trend_data)] for state in trend_emission_probs.keys()])

# row = hidden state (s,b,h), column = observation symbol (down,up,no change)

print("Trend Emission Matrix:")
print(trend_emission_matrix)


Trend Emission Matrix:
[[0.05109489 0.2189781  0.72992701]
 [0.         1.         0.        ]
 [1.         0.         0.        ]]


In [16]:
# Building Price Emission Matrix

price_symbol_counts = {}

# Iterate over the time series data of trend and price data along with the corresponding hidden states
for state, trend_symbol, price_symbol in zip(hidden_states, discrete_trend_data, discrete_price_data):
    # Update symbol counts for price data
    if state not in price_symbol_counts:
        price_symbol_counts[state] = {}
    if price_symbol not in price_symbol_counts[state]:
        price_symbol_counts[state][price_symbol] = 0
    price_symbol_counts[state][price_symbol] += 1
    
    # Update state counts
    if state not in state_counts:
        state_counts[state] = 0
    state_counts[state] += 1

# Calculate emission probabilities for price data
price_emission_probs = {}
for state in price_symbol_counts:
    price_emission_probs[state] = {}
    total_count = state_counts[state]
    for price_symbol, count in price_symbol_counts[state].items():
        price_emission_probs[state][price_symbol] = count / total_count


price_emission_matrix = np.array([[price_emission_probs[state].get(symbol, 0) for symbol in set(discrete_price_data)] for state in price_emission_probs.keys()])


print("Price Emission Matrix:")
print(price_emission_matrix)

Price Emission Matrix:
[[0.24817518 0.12408759 0.12773723]
 [0.         0.19318182 0.30681818]
 [0.31967213 0.         0.18032787]]


In [24]:
# Constructing the Transition Matrix

# row is t, column is t+1

# Construct count of transitions from one hidden state to another i.e how many times 0 becomes 1, 1 becomes 2 etc.
transition_counts = {}
for i in range(1, len(hidden_states)):
    transition = (hidden_states[i-1], hidden_states[i])
    if transition not in transition_counts:
        transition_counts[transition] = 0
    transition_counts[transition] += 1

print(transition_counts)

# Construct the transition matrix
transition_matrix = np.zeros((3, 3))
for (state1, state2), count in transition_counts.items():
    state1_index = {"buy": 0, "sell": 1, "hold": 2}[state1]
    state2_index = {"buy": 0, "sell": 1, "hold": 2}[state2]
    transition_matrix[state1_index, state2_index] = count / state_counts[state1]


print("Transition Matrix:")
print(transition_matrix)


{('hold', 'buy'): 24, ('buy', 'hold'): 25, ('hold', 'hold'): 77, ('buy', 'sell'): 14, ('sell', 'sell'): 11, ('sell', 'buy'): 15, ('buy', 'buy'): 5, ('hold', 'sell'): 36, ('sell', 'hold'): 34}
Transition Matrix:
[[0.05681818 0.15909091 0.28409091]
 [0.12295082 0.09016393 0.27868852]
 [0.08759124 0.13138686 0.2810219 ]]
